In [1]:
import json
import csv
import urllib.request, urllib.parse, urllib.error

json_file_location="C:/Users/toddw/Desktop/Python Rating Code and Files/OSAA_2025_Ratings/json_match_files/"

In [2]:
#Names that got badly parsed into too many fields, which causes problems when writing to csv files
bad_parsed_name_list=[144173,144187,161966]
bad_parsed_name_replacements={
    144173:'Ana Carcano-Blas',
    144187:'Raquel Navarro-Karla',
    161966:'Kendra Warren-Galligan'
}

In [3]:
################## Player ID replacements for players with multiple IDs *****************

player_id_replacements={
    124015:144721,
    124727:122976,
    143338:122284,
    102053:143853,
    123110:122733
    }

# Use this function at the end of the process to replace the player IDs
def replace_values_in_list(lst, replacements):
    for row in lst:
        for index in (3, 4, 5, 6):
            if row[index] in replacements:
                row[index] = replacements[row[index]]

In [4]:
rubens_d5d9_matches=[190334,193154,194320]

In [5]:
unflighted_tournament_list=[165085,175470,175391,176646,176615,176561,176440,176423,176413,176353,176241,176010,175907
                           ,175832,175785,175777,175748,175389,175384,174806,174788,174670,176856,176855,176857,176858
                           ,176859,176860
                           ,192881,196696,196700]

In [6]:
bad_match_score_list=[1054976 #too broken to fix
                     ,1260796,1266032,1279911,1279098,1279937 #fixed below      
                     ,1039201,1273579  #anomalous score (sick/injured player)
                     ,1042910  #Contains a player duplicated in the match - most likely accurate row was kept
                      #     ,929302,928245
                     ,1084012,1281724,1274863 # retirement early in match
                     ,1282980,1282988 # (0-0, 0-0, 0-1)
                     ,1285926 #try to fix later - this is Bella Plaisted in at 5th doubles vs Liberty
                     ]  


In [7]:
replaced_match_scores=[] #add new ones to the bad_match_score_list above too
corrected_row1=['1260796','4/9/2025','D2','121172','122443','121173','143733','6*0','5*7','1*0','190631','Cassia Maciejewski','Mila Switzer','Harper Heiser','Zoe Lazzara']
replaced_match_scores.append(corrected_row1)
corrected_row2=['1266032','4/11/2025','D4','168688','143732','143591','169128','6*2','6*1','0*0','191314','Vivienne Dempsey','WeiLan Nathan','Peyton Villareal','Angelica Quiett']
replaced_match_scores.append(corrected_row2)
corrected_row3=['1279911','4/18/2025','D3','166290','151925','104400','123196','6*2','7*5','0*0','192796','Ariana Barsuglia','Paige Squires','Zuzanna Bownik','Nina Vivek']
replaced_match_scores.append(corrected_row3)
corrected_row4=['1279098','4/17/2025','D3','165114','165119','124345','130719','6*4','6*1','0*0','192696','Hannah Hendricks','Vanesa Aguilar-Thompson','Hannah  Forrest','Dailey True']
replaced_match_scores.append(corrected_row4)
corrected_row5=['1279937','4/18/2025','D1','123656','105485','102463','122724','6*3','1*6','0*0','192696','Kaitlin Carter','Avalon Donnelly','Torah Womack','Alyssa Piquette']
replaced_match_scores.append(corrected_row5)




In [8]:
# Replace common bad scores with my best guess for their actual values

bad_4byte_set_cleanup_library = { #this is needed up front for the score parsing looking for the * in normal places
    "60*0":"6*0",
    "61*0":"6*1",
    "62*0":"6*2",
    "63*0":"6*3",
    "64*0":"6*4",
    "75*0":"7*5",
    "76*0":"7*6",
    "16*0":"1*6",
    "26*0":"2*6",
    "36*0":"3*6",
    "46*0":"4*6",
    "57*0":"5*7",
    "67*0":"6*7",
    "84*0":"8*4",
    "6*67":"6*7",
    "7*62":"6*2",
    "6*20":"6*2",
    "9*11":"6*7"
}


score_replacement_library = {
    "6*5":"7*6",
    "7*0":"7*6",
    "7*1":"7*6",
    "7*2":"7*6",
    "7*3":"7*6",
    "7*4":"7*6",
    "6*6":"7*6",
    "7*7":"7*6",
    "8*8":"9*8",
    "11*9":"9*8",
    "10*8":"7*6",
    "5*1":"6*1",
    "5*2":"6*2",
    "5*3":"6*3",
    "62*0":"6*2",
    "7*62":"6*2",
    "**6":"6*2",
    "61*0":"6*1",
    "9*11":"6*7"
}

score_replacement_library_set2 = {
    "7*0":"7*6",
    "7*1":"7*6",
    "7*2":"7*6",
    "7*3":"7*6",
    "7*4":"7*6",
    "6*6":"7*6",
    "7*7":"7*6",
    "8*8":"9*8",
    "11*9":"7*6",
    "10*8":"7*6",
    "12*10":"7*6",
    "13*11":"7*6",
    "5*1":"6*1",
    "5*2":"6*2",
    "5*3":"6*3",
    "9*7":"7*6",
    "9*7":"7*6",
    "2*5":"2*6",
    "6*8":"6*7",
    "8*6":"7*6",
    "8*7":"7*6",
    "61*0":"6*1",
    "**7":"6*7",
    "7*9":"6*7",
    "6*20": "6*2",
    "64*0":"6*4",
    "**1":"6*7",
    "9*11":"6*7"
}

score_replacement_library_set3 = {
    "6*6":"7*6",
    "8*8":"9*8",
    "11*9":"1*0",
    "5*1":"6*1",
    "5*2":"6*2",
    "5*3":"6*3",
    "7*0":"1*0",
    "7*1":"1*0",
    "7*2":"1*0",
    "7*3":"1*0",
    "7*4":"1*0",
    "8*5":"1*0",
    "8*3":"1*0",
    "9*7":"1*0",
    "8*6":"1*0"
}

In [9]:
#Create a file and header for storing team match results
team_match_header=list()
data0=["Team_Match_ID","Winning_Team_ID","Losing_Team_ID","Winning_Team_Name","Losing_Team_Name","Winning_Team_Win_Cnt","Losing_Team_Win_Cnt","Match_Dt","Team_Match_Summary"]
team_match_header.append(data0)

with open ('osaa_team_matches_2025.csv', 'w', newline='\n') as csvfile:
    for item in team_match_header:
        matchwriter=csv.writer(csvfile, delimiter=',',quotechar='|',quoting=csv.QUOTE_MINIMAL)
        matchwriter.writerow(item)

In [10]:
#Create a file and header for storing singles and doubles match results
indv_match_header=list()
data0=["MatchID","Match_Dt","Position","Winner1","Winner2","Loser1","Loser2","Set1","Set2","Set3","Team_Match_ID","Winner1_Name","Winner2_Name","Loser1_Name","Loser2_Name"]
indv_match_header.append(data0)

with open ('osaa_matches_2025.csv', 'w', newline='\n') as csvfile:
    for item in indv_match_header:
        matchwriter=csv.writer(csvfile, delimiter=',',quotechar='|',quoting=csv.QUOTE_MINIMAL)
        matchwriter.writerow(item)

In [11]:
# Create a list of schools with scores to loop through in later processes
schools_to_rate_list=[]
schools_to_rate = open('school_league_info.csv')
for line in schools_to_rate:
    fullline=line.strip()
    linelist=fullline.split(',')
    schoolid=linelist[0]
    schools_to_rate_list.append(schoolid)

print(schools_to_rate_list)

['74775', '74707', '74713', '74728', '74776', '74804', '75547', '124983', '7070586', '74852', '124825', '74861', '124895', '74869', '124868', '74876', '74631', '75034', '75386', '75548', '74901', '74917', '74918', '74940', '74954', '74965', '75003', '75069', '7070534', '75549', '74639', '75550', '75097', '75112', '75205', '75182', '74634', '75142', '75555', '74665', '75341', '124879', '75188', '75219', '75227', '124831', '75236', '75237', '75251', '74645', '75552', '75153', '124728', '74929', '75207', '75554', '75688', '75321', '75289', '75683', '75387', '75330', '74805', '7070582', '7070587', '75359', '75366', '75031', '75406', '75310', '75684', '75412', '75425', '75448', '75454', '75463', '75468', '75388', '75585', '75606', '7070583', '75619', '75564', '75636', '75113', '75713', '75720', '75032', '75740', '75744', '75762', '75764', '74695', '75033', '75309', '75686', '74785', '75685', '75807', '75818', '124877', '124823', '75825', '74614', '74814', '74777', '74671', '75808', '75848',

In [12]:
### This section of code has been altered for 2025 - the try/except process was added to only process schools with results
### It also means that all schools with teams on the OSAA site are now in the school_league_info.csv file

### create list to store Team Match Info ###

### If this step crashes, you'll want to see which school it was working on to research
###   So, Uncomment the the print in the loop and possibly the one at the bottom too

import json

team_match_list = []

schools_to_rate = open('school_league_info.csv')

for line in schools_to_rate:
    fullline = line.strip()
    linelist = fullline.split(',')
    schoolid = linelist[0]

    json_match_file = 'zz' + schoolid + 'match_file.json'

    try:
        with open(json_file_location + json_match_file, 'r') as myfile:
            temp = myfile.read()

        # Parse file
        info = json.loads(temp)

        # Skip processing if "meets" is missing or empty
        if "meets" not in info or not info["meets"]:
            print(f"Skipping {json_match_file}: File error - No Meets for the School")
            continue

        ### Loop through matches to get team match details ###
        for match in info["meets"]:
            team_match_id = match["id"]
            winning_team_name = match["schools"]["winners"][0]["name"].replace(',', '')
            winning_team_id = match["schools"]["winners"][0]["id"]
            winning_team_win_cnt = match["schools"]["winners"][0]["score"]

            try:
                losing_team_name = match["schools"]["losers"][0]["name"].replace(',', '')
                losing_team_id = match["schools"]["losers"][0]["id"]
                losing_team_win_cnt = match["schools"]["losers"][0]["score"]
            except (KeyError, IndexError):
                losing_team_name = ''
                losing_team_id = ''
                losing_team_win_cnt = 99

            match_datetime = match["meetDateTime"]
            match_date = match_datetime[0:10]
            team_match_id_summary = f"{winning_team_name}_def_{losing_team_name}_{winning_team_win_cnt}-{losing_team_win_cnt}_on_{match_date}"

            if str(winning_team_id) in schools_to_rate_list and str(losing_team_id) in schools_to_rate_list:
                data1 = [
                    team_match_id, winning_team_id, losing_team_id,
                    winning_team_name, losing_team_name,
                    str(winning_team_win_cnt), str(losing_team_win_cnt),
                    match_date, team_match_id_summary
                ]
                team_match_list.append(data1)

    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Skipping {json_match_file} - No match results yet")

###    for item in team_match_list:
###        print(item)

In [13]:
#dedupe the list of results and write them to the team match csv file

team_match_list_deduped = []
for x in team_match_list:
    if x not in team_match_list_deduped:
        team_match_list_deduped.append(x)

with open ('osaa_team_matches_2025.csv', 'a', newline='\n') as csvfile:
    for item in team_match_list_deduped:
        matchwriter=csv.writer(csvfile, delimiter=',',quotechar='|',quoting=csv.QUOTE_MINIMAL)
        matchwriter.writerow(item)

In [14]:
######################################  gather singles match information  ######################################

### prepare match information for scoring code ###
scoring_file_list=list()

schools_to_rate = open('school_league_info.csv')
for line in schools_to_rate:
    fullline=line.strip()
    linelist=fullline.split(',')
    schoolid=linelist[0]

    json_match_file='zz'+schoolid+'match_file.json'
    with open(json_file_location+json_match_file, 'r') as myfile:
        temp=myfile.read()
    # parse file
    info = json.loads(temp)
    print('Working on School ID: ',schoolid)

    ### prepare for match information detail file ###
    indv_match_list=list()
    data0=["Team_Match_ID","Flight","Match_ID","PlayerA_ID","PlayerB_ID","PlayerC_ID","PlayerD_ID","PlayerA_ID_in_Match","PlayerB_ID_in_Match","PlayerC_ID_in_Match","PlayerD_ID_in_Match","PlayerA_Winner_TF","PlayerA_FullName","PlayerB_FullName","PlayerC_Full_Name","PlayerD_FullName","Set1","Set2",'Set3",'"Match_Score"]
    indv_match_list.append(data0)

    # Skip processing if "meets" is missing or empty
    if "meets" not in info or not info["meets"]:
        print(f"Skipping {json_match_file}: File error - No Meets for the School")
        continue

    try:    
        
        i=0
        while i <= len(info["meets"])-1:
            s=0
            try:
                len(info["meets"][i]["matches"]["Singles"])
            except:
                print("This meet has no singles matches")
            else:
                while s <= len(info["meets"][i]["matches"]["Singles"])-1:
                    try:
                        singles_player1_real_id=info["meets"][i]["matches"]["Singles"][s]["matchTeams"][1]["players"][0]["id"]
                        singles_player0_real_id=info["meets"][i]["matches"]["Singles"][s]["matchTeams"][0]["players"][0]["id"]
                        sets_played=len(info["meets"][i]["matches"]["Doubles"][s]["sets"])
                    except:
                        print("This match has blank names or scores")
                    else:
                        team_match_id=info["meets"][i]["id"]
                        match_datetime=info["meets"][i]["meetDateTime"]
                        match_date=match_datetime[0:10]
                        flight="S"+info["meets"][i]["matches"]["Singles"][s]["flight"]
                        singles_match_id=info["meets"][i]["matches"]["Singles"][s]["id"]
                        singles_player0_id_in_match=info["meets"][i]["matches"]["Singles"][s]["matchTeams"][0]["id"]
                        singles_player0_isWinner=info["meets"][i]["matches"]["Singles"][s]["matchTeams"][0]["isWinner"]
                        singles_player0_real_id=info["meets"][i]["matches"]["Singles"][s]["matchTeams"][0]["players"][0]["id"]
                        singles_player0_fullname=info["meets"][i]["matches"]["Singles"][s]["matchTeams"][0]["players"][0]["firstName"]+" "+info["meets"][i]["matches"]["Singles"][s]["matchTeams"][0]["players"][0]["lastName"]

                        singles_player1_id_in_match=info["meets"][i]["matches"]["Singles"][s]["matchTeams"][1]["id"]
                        singles_player1_real_id=info["meets"][i]["matches"]["Singles"][s]["matchTeams"][1]["players"][0]["id"]
                        singles_player1_fullname=info["meets"][i]["matches"]["Singles"][s]["matchTeams"][1]["players"][0]["firstName"]+" "+info["meets"][i]["matches"]["Singles"][s]["matchTeams"][1]["players"][0]["lastName"]
                        setnum=0
                        set1_pre="0*0"
                        set2_pre="0*0"
                        set3_pre="0*0"

                        while setnum <= len(info["meets"][i]["matches"]["Singles"][s]["sets"])-1:
                            singles_player0_set_games=info["meets"][i]["matches"]["Singles"][s]["sets"][setnum][str(singles_player0_id_in_match)]
                            singles_player1_set_games=info["meets"][i]["matches"]["Singles"][s]["sets"][setnum][str(singles_player1_id_in_match)]
                            singles_match_set_score=str(singles_player0_set_games)+"*"+str(singles_player1_set_games)
                            set_from_json=info["meets"][i]["matches"]["Singles"][s]["sets"][setnum]["number"]
                            if set_from_json==1:
                                match_score=singles_match_set_score
                                set1_pre=singles_match_set_score
                            else:
                                match_score=match_score+" ; "+singles_match_set_score
                                if set_from_json==2:
                                    set2_pre=singles_match_set_score
                                else:
                                    if set_from_json==3:
                                        set3_pre=singles_match_set_score
                            setnum=setnum+1
                        data1=[team_match_id,flight,singles_match_id,singles_player0_real_id,None,singles_player1_real_id,None,singles_player0_id_in_match,singles_player1_id_in_match,singles_player0_isWinner,singles_player0_fullname,None,singles_player1_fullname,None,set1_pre,set2_pre,set3_pre,match_score]
                        indv_match_list.append(data1)

                        #Assign the correct winner
                        if singles_player0_isWinner==True:
                            winner1=singles_player0_real_id
                            winner1_name=singles_player0_fullname
                            loser1=singles_player1_real_id
                            loser1_name=singles_player1_fullname
                        else:
                            winner1=singles_player1_real_id
                            winner1_name=singles_player1_fullname
                            loser1=singles_player0_real_id
                            loser1_name=singles_player0_fullname
                        #Clean out --- None None set scores
                        if set3_pre=="None*None":
                            set3_pre="0*0"
                        if set2_pre=="None*None":
                            set2_pre="0*0"
                        if set1_pre=="None*None":
                            set1_pre="0*0"



                        # clean up the massively wrong set scores
                        if set1_pre in bad_4byte_set_cleanup_library:
                            set1_pre = bad_4byte_set_cleanup_library[set1_pre]
                        if set2_pre in bad_4byte_set_cleanup_library:
                            set2_pre = bad_4byte_set_cleanup_library[set2_pre]
                        if set3_pre in bad_4byte_set_cleanup_library:
                            set3_pre = bad_4byte_set_cleanup_library[set3_pre]





                        #Reverse the Set Scores if they are backwards
                        #but first move sets left if they put in sets 2 and 3 instead of 1 and 2
                        if set1_pre=='0*0' and set2_pre!='0*0':
                            set1_pre=set2_pre
                            set2_pre=set3_pre
                            set3_pre='0*0'

                        if set3_pre!="0*0" and len(set3_pre)==3:
                            if set3_pre[2]<set3_pre[0]:
                                set1=set1_pre
                                set2=set2_pre
                                set3=set3_pre
                            else:
                                set1=set1_pre[2]+"*"+set1_pre[0]
                                set2=set2_pre[2]+"*"+set2_pre[0]
                                set3=set3_pre[2]+"*"+set3_pre[0]
                        elif len(set3_pre)==4 and set3_pre[2]=='*':
                            set1=set1_pre
                            set2=set2_pre
                            set3='1*0'
                        elif len(set3_pre)==4 and set3_pre[1]=='*':
                            set1=set1_pre[2]+"*"+set1_pre[0]
                            set2=set2_pre[2]+"*"+set2_pre[0]
                            set3='1*0'
                        elif len(set3_pre)==5:
                            if set3_pre[3:]<set3_pre[0:2]:
                                set1=set1_pre
                                set2=set2_pre
                                set3='1*0'
                            else:
                                set1=set1_pre[2]+"*"+set1_pre[0]
                                set2=set2_pre[2]+"*"+set2_pre[0]
                                set3='1*0'
                        else:
                            if set2_pre!="0*0":
                                if set2_pre[2]<set2_pre[0]:
                                    set1=set1_pre
                                    set2=set2_pre
                                    set3=set3_pre
                                else:
                                    set1=set1_pre[2]+"*"+set1_pre[0]
                                    set2=set2_pre[2]+"*"+set2_pre[0]
                                    set3=set3_pre[2]+"*"+set3_pre[0]
                            else:
                                if set1_pre!="0*0":
                                    if set1_pre[2]<set1_pre[0]:
                                        set1=set1_pre
                                        set2=set2_pre
                                        set3=set3_pre
                                    else:
                                        set1=set1_pre[2]+"*"+set1_pre[0]
                                        set2=set2_pre[2]+"*"+set2_pre[0]
                                        set3=set3_pre[2]+"*"+set3_pre[0]
                        if set1 in score_replacement_library:
                            set1 = score_replacement_library[set1]
                        if set2 in score_replacement_library_set2:
                            set2 = score_replacement_library_set2[set2]
                        if set3 in score_replacement_library_set3:
                            set3 = score_replacement_library_set3[set3]
                        if team_match_id in unflighted_tournament_list:
                            flight='SX'
                        scoring_file_record=[singles_match_id,match_date,flight,winner1,None,loser1,None,set1,set2,set3,team_match_id,winner1_name,None,loser1_name,None]
                        if set1 != '2*0' and singles_match_id not in bad_match_score_list:
                            scoring_file_list.append(scoring_file_record)
                    s=s+1
            i=i+1

        #for item in indv_match_list:
        #    print(item)

        #for item in scoring_file_list:
        #    print(item)

        ######################################  gather doubles match information  ######################################


        i=0
        while i <= len(info["meets"])-1:
            s=0
            try:
                len(info["meets"][i]["matches"]["Doubles"])
            except:
                print("This meet has no doubles matches")
            else:
                while s <= len(info["meets"][i]["matches"]["Doubles"])-1:
                    try:
                        doubles_player0_real_id=info["meets"][i]["matches"]["Doubles"][s]["matchTeams"][0]["players"][0]["id"]
                        doubles_player1_real_id=info["meets"][i]["matches"]["Doubles"][s]["matchTeams"][0]["players"][1]["id"]
                        doubles_player2_real_id=info["meets"][i]["matches"]["Doubles"][s]["matchTeams"][1]["players"][0]["id"]
                        doubles_player3_real_id=info["meets"][i]["matches"]["Doubles"][s]["matchTeams"][1]["players"][1]["id"]
                        sets_played=len(info["meets"][i]["matches"]["Doubles"][s]["sets"])
                    except:
                        print("This match has blank names or scores")
                    else:
                        team_match_id=info["meets"][i]["id"]
                        match_datetime=info["meets"][i]["meetDateTime"]
                        match_date=match_datetime[0:10]
                        flight="D"+info["meets"][i]["matches"]["Doubles"][s]["flight"]
                        doubles_match_id=info["meets"][i]["matches"]["Doubles"][s]["id"]
                        doubles_player0_isWinner=info["meets"][i]["matches"]["Doubles"][s]["matchTeams"][0]["isWinner"]

                        doubles_team0_id_in_match=info["meets"][i]["matches"]["Doubles"][s]["matchTeams"][0]["id"]
                        doubles_player0_real_id=info["meets"][i]["matches"]["Doubles"][s]["matchTeams"][0]["players"][0]["id"]
                        doubles_player0_fullname=info["meets"][i]["matches"]["Doubles"][s]["matchTeams"][0]["players"][0]["firstName"]+" "+info["meets"][i]["matches"]["Doubles"][s]["matchTeams"][0]["players"][0]["lastName"]
                        doubles_player1_real_id=info["meets"][i]["matches"]["Doubles"][s]["matchTeams"][0]["players"][1]["id"]
                        doubles_player1_fullname=info["meets"][i]["matches"]["Doubles"][s]["matchTeams"][0]["players"][1]["firstName"]+" "+info["meets"][i]["matches"]["Doubles"][s]["matchTeams"][0]["players"][1]["lastName"]

                        doubles_team1_id_in_match=info["meets"][i]["matches"]["Doubles"][s]["matchTeams"][1]["id"]
                        doubles_player2_real_id=info["meets"][i]["matches"]["Doubles"][s]["matchTeams"][1]["players"][0]["id"]
                        doubles_player2_fullname=info["meets"][i]["matches"]["Doubles"][s]["matchTeams"][1]["players"][0]["firstName"]+" "+info["meets"][i]["matches"]["Doubles"][s]["matchTeams"][1]["players"][0]["lastName"]
                        doubles_player3_real_id=info["meets"][i]["matches"]["Doubles"][s]["matchTeams"][1]["players"][1]["id"]
                        doubles_player3_fullname=info["meets"][i]["matches"]["Doubles"][s]["matchTeams"][1]["players"][1]["firstName"]+" "+info["meets"][i]["matches"]["Doubles"][s]["matchTeams"][1]["players"][1]["lastName"]

                        setnum=0
                        set1_pre="0*0"
                        set2_pre="0*0"
                        set3_pre="0*0"



                        # clean up the massively wrong set scores
                        if set1_pre in bad_4byte_set_cleanup_library:
                            set1_pre = bad_4byte_set_cleanup_library[set1_pre]
                        if set2_pre in bad_4byte_set_cleanup_library:
                            set2_pre = bad_4byte_set_cleanup_library[set2_pre]
                        if set3_pre in bad_4byte_set_cleanup_library:
                            set3_pre = bad_4byte_set_cleanup_library[set3_pre]



                        while setnum <= len(info["meets"][i]["matches"]["Doubles"][s]["sets"])-1:
                            doubles_team0_set_games=info["meets"][i]["matches"]["Doubles"][s]["sets"][setnum][str(doubles_team0_id_in_match)]
                            doubles_team1_set_games=info["meets"][i]["matches"]["Doubles"][s]["sets"][setnum][str(doubles_team1_id_in_match)]
                            doubles_match_set_score=str(doubles_team0_set_games)+"*"+str(doubles_team1_set_games)
                            set_from_json=info["meets"][i]["matches"]["Doubles"][s]["sets"][setnum]["number"]
                            if set_from_json==1:
                                match_score=doubles_match_set_score
                                set1_pre=doubles_match_set_score
                            else:
                                match_score=match_score+" ; "+doubles_match_set_score
                                if set_from_json==2:
                                    set2_pre=doubles_match_set_score
                                else:
                                    if set_from_json==3:
                                        set3_pre=doubles_match_set_score
                            setnum=setnum+1
                        data1=[team_match_id,flight,doubles_match_id,doubles_player0_real_id,doubles_player1_real_id,doubles_player2_real_id,doubles_player3_real_id,doubles_team0_id_in_match,doubles_team1_id_in_match,doubles_player0_isWinner,doubles_player0_fullname,doubles_player1_fullname,doubles_player2_fullname,doubles_player3_fullname,set1_pre,set2_pre,set3_pre,match_score]
                        indv_match_list.append(data1)

                        #Assign the correct winner
                        if doubles_player0_isWinner==True:
                            winner1=doubles_player0_real_id
                            winner1_name=doubles_player0_fullname
                            winner2=doubles_player1_real_id
                            winner2_name=doubles_player1_fullname
                            loser1=doubles_player2_real_id
                            loser1_name=doubles_player2_fullname
                            loser2=doubles_player3_real_id
                            loser2_name=doubles_player3_fullname
                        else:
                            winner1=doubles_player2_real_id
                            winner1_name=doubles_player2_fullname
                            winner2=doubles_player3_real_id
                            winner2_name=doubles_player3_fullname
                            loser1=doubles_player0_real_id
                            loser1_name=doubles_player0_fullname
                            loser2=doubles_player1_real_id
                            loser2_name=doubles_player1_fullname
                        #Clean out --- None None set scores
                        if set3_pre=="None*None":
                            set3_pre="0*0"
                        if set2_pre=="None*None":
                            set2_pre="0*0"
                        if set1_pre=="None*None":
                            set1_pre="0*0"

                        #Reverse the Set Scores if they are backwards
                        #but first move sets left if they put in sets 2 and 3 instead of 1 and 2
                        if set1_pre=='0*0' and set2_pre!='0*0':
                            set1_pre=set2_pre
                            set2_pre=set3_pre
                            set3_pre='0*0'

                        if set3_pre!="0*0" and len(set3_pre)==3:
                            if set3_pre[2]<set3_pre[0]:
                                set1=set1_pre
                                set2=set2_pre
                                set3=set3_pre
                            else:
                                set1=set1_pre[2]+"*"+set1_pre[0]
                                set2=set2_pre[2]+"*"+set2_pre[0]
                                set3=set3_pre[2]+"*"+set3_pre[0]
                        elif len(set3_pre)==4 and set3_pre[2]=='*':
                            set1=set1_pre
                            set2=set2_pre
                            set3='1*0'
                        elif len(set3_pre)==4 and set3_pre[1]=='*':
                            set1=set1_pre[2]+"*"+set1_pre[0]
                            set2=set2_pre[2]+"*"+set2_pre[0]
                            set3='1*0'
                        elif len(set3_pre)==5:
                            if set3_pre[3:]<set3_pre[0:2]:
                                set1=set1_pre
                                set2=set2_pre
                                set3='1*0'
                            else:
                                set1=set1_pre[2]+"*"+set1_pre[0]
                                set2=set2_pre[2]+"*"+set2_pre[0]
                                set3='1*0'
                        else:
                            if set2_pre!="0*0":
                                if set2_pre[2]<set2_pre[0]:
                                    set1=set1_pre
                                    set2=set2_pre
                                    set3=set3_pre
                                else:
                                    set1=set1_pre[2]+"*"+set1_pre[0]
                                    set2=set2_pre[2]+"*"+set2_pre[0]
                                    set3=set3_pre[2]+"*"+set3_pre[0]
                            else:
                                if set1_pre!="0*0":
                                    if set1_pre[2]<set1_pre[0]:
                                        set1=set1_pre
                                        set2=set2_pre
                                        set3=set3_pre
                                    else:
                                        set1=set1_pre[2]+"*"+set1_pre[0]
                                        set2=set2_pre[2]+"*"+set2_pre[0]
                                        set3=set3_pre[2]+"*"+set3_pre[0]
                        if set1 in score_replacement_library:
                            set1 = score_replacement_library[set1]
                        if set2 in score_replacement_library_set2:
                            set2 = score_replacement_library_set2[set2]
                        if set3 in score_replacement_library_set3:
                            set3 = score_replacement_library_set3[set3]
                        if team_match_id in unflighted_tournament_list:
                            flight='DX'
                        scoring_file_record=[doubles_match_id,match_date,flight,winner1,winner2,loser1,loser2,set1,set2,set3,team_match_id,winner1_name,winner2_name,loser1_name,loser2_name]


                        if scoring_file_record[3] in bad_parsed_name_list:
                            scoring_file_record[11]=bad_parsed_name_replacements[scoring_file_record[3]]
                            print(scoring_file_record)
                        if scoring_file_record[4] in bad_parsed_name_list:
                            scoring_file_record[12]=bad_parsed_name_replacements[scoring_file_record[4]]
                            print(scoring_file_record)
                        if scoring_file_record[5] in bad_parsed_name_list:
                            scoring_file_record[13]=bad_parsed_name_replacements[scoring_file_record[5]]
                            print(scoring_file_record)
                        if scoring_file_record[6] in bad_parsed_name_list:
                            scoring_file_record[14]=bad_parsed_name_replacements[scoring_file_record[6]]
                            print(scoring_file_record)




                        if set1 != '2*0' and doubles_match_id not in bad_match_score_list:
                            scoring_file_list.append(scoring_file_record)
                    s=s+1
            i=i+1

    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Skipping {json_match_file} - No match results yet")
        
    #for item in indv_match_list:
        #print(item)

    #for item in scoring_file_list:
        #print(item)


Working on School ID:  74775
[1241962, '2025-03-18', 'D2', 100087, 143247, 144173, 144174, '6*4', '7*6', '0*0', 188012, 'Emily Aguilar-Santiago', 'Perla Sanchez', 'Ana Carcano-Blas', ' Ashley Castañon']
Working on School ID:  74707
This match has blank names or scores
This match has blank names or scores
This match has blank names or scores
This match has blank names or scores
This match has blank names or scores
This match has blank names or scores
This match has blank names or scores
This match has blank names or scores
This match has blank names or scores
This match has blank names or scores
This meet has no doubles matches
Working on School ID:  74713
This match has blank names or scores
This match has blank names or scores
This match has blank names or scores
Working on School ID:  74728
This match has blank names or scores
This match has blank names or scores
This match has blank names or scores
This match has blank names or scores
This match has blank names or scores
This match 

Working on School ID:  75412
This match has blank names or scores
This match has blank names or scores
This match has blank names or scores
This match has blank names or scores
[1283948, '2025-04-21', 'D2', 141166, 144354, 161966, 162344, '6*2', '6*1', '0*0', 193194, 'Samatha` Toll', 'Emily Cornell', 'Kendra Warren-Galligan', 'Megan Daniels']
This match has blank names or scores
This match has blank names or scores
This match has blank names or scores
Working on School ID:  75425
This match has blank names or scores
This match has blank names or scores
This match has blank names or scores
This match has blank names or scores
This match has blank names or scores
[1317782, '2025-05-06', 'D2', 141671, 141677, 161966, 162344, '6*1', '6*3', '0*0', 197224, 'Isabella Bernier-Alexander', 'Hannah  Morioka', 'Kendra Warren-Galligan', 'Megan Daniels']
[1293168, '2025-04-24', 'D3', 141459, 141460, 161966, 162344, '6*2', '6*4', '0*0', 194308, 'Alejandra Reyes-Carranza', 'Luz Maria Reyes-Carranza', 

In [15]:
for item in scoring_file_list:
    if item[2] == 'D6':
        print(item)
#print(rubens_d5d9_matches)

[1325431, '2025-05-08', 'D6', 108618, 108619, 122995, 144170, '6*2', '6*1', '0*0', 197653, 'Jordan Montoya', 'McKenzie Howell', 'Mars Gonzalez-Resendiz', 'Leila Mooney']
[1313425, '2025-05-02', 'D6', 123031, 165114, 104328, 144174, '6*0', '6*0', '0*0', 196649, 'Abby Nash', 'Hannah Hendricks', 'Ashlyn Molen', ' Ashley Castañon']
[1293328, '2025-04-24', 'D6', 164941, 164942, 144173, 164944, '7*6', '6*4', '0*0', 194320, 'Ava Dodson', 'Sofie Fox', 'Ana Carcano-Blas', 'Miriam Plopper']
[1285922, '2025-04-17', 'D6', 104328, 144174, 144330, 145408, '6*0', '6*0', '0*0', 193154, 'Ashlyn Molen', ' Ashley Castañon', 'Ruby Miller', 'Autumn Schnathorst']
[1258425, '2025-04-08', 'D6', 164375, 164382, 104328, 164944, '6*0', '6*2', '0*0', 190334, 'Anna Bigham', 'Emi Mackura', 'Ashlyn Molen', 'Miriam Plopper']
[1293328, '2025-04-24', 'D6', 164941, 164942, 144173, 164944, '7*6', '6*4', '0*0', 194320, 'Ava Dodson', 'Sofie Fox', 'Ana Carcano-Blas', 'Miriam Plopper']
[1258425, '2025-04-08', 'D6', 164375, 1

In [16]:
for item in scoring_file_list:
    if item[10] in rubens_d5d9_matches:
        if item[2] == 'D5':
            item[2] = 'D1'
        elif item[2] == 'D6':
            item[2] = 'D2'
        elif item[2] == 'D7':
            item[2] = 'D3'
        elif item[2] == 'D8':
            item[2] = 'D4'
        elif item[2] == 'D9':
            item[2] = 'D5'

In [17]:
# dedupe the match list
indv_match_list_deduped = []
for x in scoring_file_list:
    if x not in indv_match_list_deduped:
        indv_match_list_deduped.append(x)

for x in replaced_match_scores:
    indv_match_list_deduped.append(x)

In [18]:
# Use the replacement function from up top
# Some players have multiple player_ids and appear as two players - this consolidates them

replace_values_in_list(indv_match_list_deduped, player_id_replacements)



In [19]:
#Write out the match file
with open ('osaa_matches_2025.csv', 'a', newline='\n') as csvfile:
    for item in indv_match_list_deduped:
        matchwriter=csv.writer(csvfile, delimiter=',',quotechar='|',quoting=csv.QUOTE_MINIMAL)
        matchwriter.writerow(item)

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
df_matches = pd.read_csv('osaa_matches_2025.csv', encoding='unicode_escape', header=0)
df_matches.head()


,MatchID,Match_Dt,Position,Winner1,Winner2,Loser1,Loser2,Set1,Set2,Set3,Team_Match_ID,Winner1_Name,Winner2_Name,Loser1_Name,Loser2_Name
0,1323232,2025-05-08,S1,121899,NaN,164417,NaN,6*1,6*0,0*0,197931,Sabrina Larsen,NaN,Danielle Lee,NaN
1,1323236,2025-05-08,S2,99552,NaN,123523,NaN,6*0,6*0,0*0,197931,Kate Paine,NaN,Jasmine Arteaga,NaN
2,1323237,2025-05-08,S3,121904,NaN,123521,NaN,6*0,6*0,0*0,197931,Anna Bucher,NaN,Melissa Alonso,NaN
3,1323243,2025-05-08,S4,141668,NaN,123524,NaN,6*0,6*1,0*0,197931,Ivy Brenneke,NaN,Bailey Avalos,NaN
4,1317709,2025-05-06,S1,100768,NaN,164417,NaN,6*0,6*0,0*0,197217,Giselli Reschke,NaN,Danielle Lee,NaN


In [22]:
print("Values in Set1")
df_matches['Set1'].value_counts()

Values in Set1


Set1
6*0    1530
6*1    1254
6*2     894
6*3     730
6*4     604
7*5     256
7*6     187
4*6     119
3*6     111
8*0      97
2*6      88
8*2      78
8*1      77
5*7      63
8*4      62
6*7      45
8*3      44
8*6      42
8*5      34
1*6      30
9*7      22
0*6      13
9*8      11
8*7       4
5*0       3
0*0       2
4*1       2
4*2       2
4*3       1
Name: count, dtype: int64

In [23]:
print("Values in Set2")
df_matches['Set2'].value_counts()

Values in Set2


Set2
6*0    1546
6*1    1231
6*2     986
6*3     701
6*4     618
0*0     502
7*5     224
7*6     195
4*6     112
3*6      74
2*6      59
5*7      49
6*7      36
1*6      31
0*6      19
1*0       4
3*2       3
3*0       2
4*3       2
2*1       2
3*1       2
2*0       1
5*5       1
5*0       1
6*5       1
4*2       1
1*1       1
4*1       1
Name: count, dtype: int64

In [24]:
print("Values in Set3")
df_matches['Set3'].value_counts()

Values in Set3


Set3
0*0    5546
1*0     508
6*4      83
6*2      60
6*1      57
6*3      51
7*6      37
7*5      35
6*0      27
8*1       1
Name: count, dtype: int64

In [25]:
#Modify the code below to research possibly bad set scores
df_matches[df_matches['Set2'] == '**1']

,MatchID,Match_Dt,Position,Winner1,Winner2,Loser1,Loser2,Set1,Set2,Set3,Team_Match_ID,Winner1_Name,Winner2_Name,Loser1_Name,Loser2_Name


In [26]:
cleared_dup_match_list=[188401,188815,188826,189248,189309,189349]

In [27]:
def check_duplicate_names(group):
    winners = group[['Winner1_Name', 'Winner2_Name']].values.flatten()
    losers = group[['Loser1_Name', 'Loser2_Name']].values.flatten()
    all_values = list(winners) + list(losers)
    all_values = [x for x in all_values if pd.notna(x)]
    duplicated_values = [x for x in all_values if all_values.count(x) > 1]
    return len(duplicated_values) > 0, duplicated_values

# Group by 'Team_Match_ID' and apply the function
duplicates_info = df_matches.groupby('Team_Match_ID').apply(check_duplicate_names)

# Get the Team_Match_IDs and duplicated values
duplicated_info = duplicates_info[duplicates_info.apply(lambda x: x[0])]

for team_match_id, (has_duplicates, duplicated_values) in duplicated_info.items():
    print(f"Team_Match_ID: {team_match_id}, Duplicated Values: {duplicated_values}")

Team_Match_ID: 188401, Duplicated Values: ['Paige Cohee', 'Allie Darland', 'Paige Cohee', 'Allie Darland']
Team_Match_ID: 188815, Duplicated Values: ['Vicky de la Torre', 'Brionna Serrano', 'Vicky de la Torre', 'Brionna Serrano', 'Miriam Patricio', 'Yuridia Calvillo', 'Keltie Rietmann', 'Keltie Rietmann', 'Miriam Patricio', 'Yuridia Calvillo']
Team_Match_ID: 188826, Duplicated Values: ['Alexandria Wee', 'Evelyn Bailey', 'Evelyn Bailey', 'Alexandria Wee']
Team_Match_ID: 189215, Duplicated Values: ['Isabell Taylor', 'Isabell Taylor']
Team_Match_ID: 189248, Duplicated Values: ['Payton Fontes', 'Abbeigh Cornellson-Vanderstoel', 'Payton Fontes', 'Abbeigh Cornellson-Vanderstoel']
Team_Match_ID: 189309, Duplicated Values: ['Elizabeth Timm', 'Elizabeth Timm']
Team_Match_ID: 189483, Duplicated Values: ['Darla Tran', 'Darla Tran']
Team_Match_ID: 189582, Duplicated Values: ['Grace  Roddan', 'Grace  Roddan']
Team_Match_ID: 189630, Duplicated Values: ['Emma Goodenough', 'Bailey Coelsch', 'Emma Good

In [28]:
#research duplicate names
df_matches[df_matches['Team_Match_ID']==192074]


,MatchID,Match_Dt,Position,Winner1,Winner2,Loser1,Loser2,Set1,Set2,Set3,Team_Match_ID,Winner1_Name,Winner2_Name,Loser1_Name,Loser2_Name
309,1273554,2025-04-15,S1,142942,NaN,101432,NaN,6*4,7*6,0*0,192074,Elizabeth Timm,NaN,Avery Lewis,NaN
310,1273559,2025-04-15,S2,99025,NaN,142332,NaN,6*0,6*0,0*0,192074,Isabel Cunningham,NaN,Norma Rodriguez-Anzaldo,NaN
311,1273564,2025-04-15,S3,142945,NaN,122378,NaN,8*0,0*0,0*0,192074,Abigail Cunningham,NaN,Bryanna Ochoa,NaN
350,1273570,2025-04-15,D1,99026,142944.0,122374,122376.0,6*1,6*1,0*0,192074,Olivia Jacoby,Kaitlyn Schwin,Isaira Diaz,Ava Arredondo
351,1273575,2025-04-15,D2,142954,142955.0,141459,141460.0,6*1,6*0,0*0,192074,Emylia Myer,Jayden Whitford,Alejandra Reyes-Carranza,Luz Maria Reyes-Carranza
